In [110]:
import pandas as pd
import faiss
import requests
import numpy as np
import os

Load Index

In [111]:
if os.path.exists('realindex'):
  index = faiss.read_index('realindex')
  print("Index Loaded") 
else:
  dim = 3072
  index = faiss.IndexFlatL2(dim)
  print("Index Created")

Index Loaded


Create text representation

In [112]:
def text_rep(row):
  text_rep = f"""Title: {row['Title']}
Publication Date: {row['Publication Date']}
Keywords: {row['Keywords']}
Abstract: {row['Abstract']}
Subject Areas: {row['Subject Areas']}
"""
  return text_rep

In [113]:
datasets = ['./Database-files/2018.csv', './Database-files/2019.csv', './Database-files/2020.csv', './Database-files/2021.csv', './Database-files/2022.csv', './Database-files/2023.csv', './Database-files/final5_arxiv_articles.csv']
df = pd.DataFrame()
for dataset in datasets:
  temp_df = pd.read_csv(dataset)
  df = pd.concat([df, temp_df], ignore_index=True)

df['text_representation'] = df.apply(text_rep, axis=1)

In [114]:
df.head()['text_representation']

0    Title: Recent developments in bifunctional air...
1    Title: The benefit of punishment sensitivity o...
2    Title: Are endocrine disrupting compounds envi...
3    Title: Probabilistic regular grammar inference...
4    Title: Optimization of cinnamon oil nanoemulsi...
Name: text_representation, dtype: object

In [115]:
print(len(df))

17438


Create and Store Embeddings

In [116]:
#X = np.empty((len(df['text_representation']), dim), dtype = 'float32')

In [117]:
def embed(text_rep):
  res = requests.post('http://localhost:11434/api/embeddings', 
                      json = {
                        'model': 'llama3.2',
                        'prompt': text_rep
                      })
  return np.array([res.json()['embedding']], dtype='float32')

In [118]:
# for i, text_rep in enumerate(df['text_representation']):
#   if i%100 == 0:
#     print(i)
#   X[i] = embed(text_rep)
  
# index.add(X)

In [119]:
# faiss.write_index(index, 'index')

Testing

In [120]:
print(df[df.Title.str.contains('bifunctional air electrodes ')]['text_representation'].to_string())

0    Title: Recent developments in bifunctional air...


In [121]:
favorite_paper = df.iloc[0]

In [122]:
test = """Title: Advances in bifunctional electrocatalysts for efficient unitized regenerative proton exchange membrane fuel cells
Publication Date: 2021-08-10
Keywords: Electrocatalysts, Bifunctional electrodes, Oxygen reduction reaction, Oxygen evolution reaction, PEM fuel cells, Hydrogen production, Energy conversion, Electrochemical performance, Catalyst design, Regenerative fuel cells
Abstract: Unitized regenerative proton exchange membrane fuel cells (UR-PEMFCs) have shown promise as a sustainable energy solution for both energy storage and conversion. However, the performance of these systems is heavily dependent on the efficiency of the air electrodes, specifically the bifunctional electrocatalysts that mediate both the oxygen reduction reaction (ORR) and the oxygen evolution reaction (OER). This paper reviews recent developments in bifunctional catalysts for UR-PEMFCs, with a focus on the design and optimization of electrocatalyst materials that enhance both reactions' efficiency. The work highlights the use of non-precious metal catalysts, the role of hybrid materials, and the advances in electrode architectures aimed at improving long-term stability and reducing costs. Recent studies demonstrate that transition metal-based catalysts offer significant potential in improving the round-trip efficiency of UR-PEMFCs, with efficiencies exceeding 60% at moderate current densities.
Subject Areas: Renewable Energy, Energy Conversion and Storage, Fuel Cell Technology, Materials Science, Electrochemistry, Environmental Sustainability
"""
test_authors = ['Jienkulsawad P.', 'Saebea D.', 'Patcharavorachot Y.', 'Kheawhom S.', 'Arpornwichanop A.']

In [123]:
def faiss_search(index, query):
  distances, indices = index.search(embed(query), 10)
  print(distances[0].max())
  return distances[0], indices[0] #We only have 1 query

In [124]:
def count_authors_matches(list1,list2):
  return sum([author in list1 for author in list2])

In [125]:
def weighted_scores(indices, distances, query_authors, df):
    results = []
    for index, distance in zip(indices, distances):
        candidate_paper = df.iloc[index]
        authors = candidate_paper['Authors']
        matches_amount = count_authors_matches(authors, query_authors)
        similarity_score = (1 - distance/distances.max())
        weighted_scores = similarity_score * (1.1 ** matches_amount)
        
        results.append((index, candidate_paper['Title'], weighted_scores))
    
    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return sorted_results


In [126]:
def recommend(index, query, query_authors, df):
  distances, indices = faiss_search(index, query)
  best_matches = weighted_scores(indices, distances, query_authors, df)
  best_indices = [x[0] for x in best_matches]
  best_matches_df = df.iloc[best_indices]
  for _,match in best_matches_df.iterrows():
     print(f"{match['text_representation']}Authors: {match['Authors']}\n")

In [127]:
best_matches = recommend(index, test, test_authors, df)

582.0587
Title: Lightweight alkali-induced lattice-dynamical stability and energy storage mechanism of quaternary Mg-based hydrides under pressure effect
Publication Date: 2022-08-19
Keywords: Ab initio, Ambient pressures, Complex hydrides, Diffusion kinetics, Dynamical stability, Hydrogen release, Mg-based hydrides, Phonon dispersion relations, Positive mode, Storage mechanism, Complex hydride, Diffusion kinetics, Hydrogen release, Phase stability
Abstract: New-discovery compositions of the lightweight quaternary Mg-based hydrides at ambient pressure are found as NaLiMgH4, NaLiMg2H6, and Na3LiMg4H12, which their stabilities were determined by all positive modes of the phonon dispersion relation. The varying compositions of Na-Li-Mg-H were analyzed using the ab initio random structure searching technique and affirmed by mechanical stability and Born stability criteria. Besides, the effects of pressure on stable structures are considered also. High-gravimetric hydrogen capacity introduc